## 1. Монета (10 баллов)

Крупье выбирает монету из двух в начале с равной вероятностью (0.5). 

Вероятности выпадения орла и решки на правильной монете - одинаковые (0.5).

Вероятность выпадения орла на неправильной монете - 0.9, решки - 0.1.

Крупье начинает подкидывать монету и выпадают только орлы.

После какого выпадения можно с уверенностью 95% говорить, что крупье выбрал неправильную монету.

### Решение

Вероятность подкинуть $k$ орлов подряд в неправильной монете:
$$P(O_k|L) = 0.9^k$$

Вероятность подкинуть $k$ орлов подряд по формуле полной вероятности:
$$P(O_k) = 0.5 \cdot 0.5^k + 0.5 \cdot 0.9^k$$

По формуле Байеса:
$$P(L|O_k) = \frac {P(O_k|L) P(L)} {P(O_k)}$$

Нужно найти такое минимальное $k$, что $P(L|O_k) \geq 0.95$:
$$ \frac {0.9^k 0.5} {0.5 \cdot 0.5^k + 0.5 \cdot 0.9^k} \geq 0.95$$
$$ 0.9^k 0.5 {0.5 \cdot 0.5^k + 0.5 \cdot 0.9^k} \geq 0.95$$
$$k \geq 5.009$$
$$k = \lceil 5.009 \rceil = 6$$

## 1. PAM (10 баллов)

Выведите формулу для коэффициентов при возведении в n-ую степень следующей матрицы:  
$$PAM = \begin{bmatrix}
    1-3a & a & a & a\\
    a & 1-3a & a & a\\
    a & a & 1-3a & a\\
    a & a & a & 1-3a\\
\end{bmatrix}$$

### Решение

$$PAM^n = \begin{bmatrix}
    - \frac {-1^{n}} 4 - \frac 3 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n\\
    \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & - \frac {-1^{n}} 4 - \frac 3 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n\\
    \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & - \frac {-1^{n}} 4 - \frac 3 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n\\
    \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & \frac 1 4 - \frac {-1^n} 4 (4a-1)^n & - \frac {-1^{n}} 4 - \frac 3 4 (4a-1)^n\\
\end{bmatrix}$$

## Алгоритм Витерби + FB (22 балла)
Реализуйте алгоритмы Витерби (выводящий по последовательности открытых состояний наиболее вероятную последовательность скрытых) 
и FB (выводящий суммарные вероятности для состояний по позициям). 

Последовательность для обоих тестов: ОРОРОРООРРРРРРРРРРОООООООО

Начальные вероятности (π) в обоих тестах - 0.5.

In [1]:
import numpy as np

In [2]:
def viterbi(a, b, pi, seq):
    len_t, k = len(seq), len(b)
    pss = np.zeros((k, len_t))
    prev = np.zeros((k, len_t)).astype(int)
    
    # algorithm
    for j in range(k):
        pss[j, 0] = b[j][seq[0]] * pi[j]
        
    for t in range(1, len_t):
        for j in range(k):
            tmps = [pss[i, t - 1] * a[i, j] * b[j][seq[t]] for i in range(k)]
            pss[j, t] = max(tmps)
            prev[j, t] = np.argmax(tmps)
    
    #answer recovery
    cur_k = np.argmax([pss[i, len_t - 1] for i in range(k)])
    cur_t = len_t - 1
    ans = [cur_k]
    while cur_t > 0:
        cur_k = prev[cur_k, cur_t]
        ans.append(cur_k)
        cur_t -= 1

    return ans[::-1]

pretty_coin_ans_replaces = ['+', '-']
pretty_coin_ans = lambda ans: ''.join(map(lambda a: pretty_coin_ans_replaces[a], ans))

In [3]:
def forward_backward(a, b, pi, seq):
    len_t, k = len(seq), len(b)
    alpha, betta = np.zeros((k, len_t)), np.ones((k, len_t))
    
    for j in range(k):
        alpha[j, 0] = b[j][seq[0]] * pi[j]
        
    for t in range(1, len_t):
        for j in range(k):
            alpha[j, t] = sum([alpha[i, t - 1] * a[i, j] * b[j][seq[t]] for i in range(k)])
            
    for t in reversed(range(len_t - 1)):
        for i in range(k):
            betta[i, t] = sum([betta[j, t + 1] * a[i, j] * b[j][seq[t + 1]] for j in range(k)])
    
    sum_alpha_i_T = np.sum([alpha[i, len_t - 1] for i in range (k)])
    return alpha * betta / sum_alpha_i_T
    

### Тест 1

a11 = 0.8  
a12 = 0.2  
a22 = 0.8  
a21 = 0.2  
b1(O) = 0.5  
b1(Р) = 0.5  
b2(О) = 0.1  
b2(Р) = 0.9

In [4]:
a = np.array([
    [0.8, 0.2],
    [0.2, 0.8]
])

b = [
    {'O': 0.5, 'P':0.5},
    {'O': 0.1, 'P':0.9}
]

seq = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

pi = [0.5, 0.5]

v1 = viterbi(a, b, pi, seq)
print(seq)
print(pretty_coin_ans(v1))
print(forward_backward(a, b, pi, seq))

OPOPOPOOPPPPPPPPPPOOOOOOOO
++++++++----------++++++++
[[0.85956173 0.76609555 0.8783159  0.77939405 0.88686798 0.79512375
  0.91061422 0.85999559 0.45699127 0.27318414 0.18984842 0.15315539
  0.13940407 0.13971214 0.15427858 0.19251224 0.27910923 0.47000478
  0.88850328 0.96461882 0.97845654 0.98093931 0.98120371 0.98022273
  0.97438637 0.94221681]
 [0.14043827 0.23390445 0.1216841  0.22060595 0.11313202 0.20487625
  0.08938578 0.14000441 0.54300873 0.72681586 0.81015158 0.84684461
  0.86059593 0.86028786 0.84572142 0.80748776 0.72089077 0.52999522
  0.11149672 0.03538118 0.02154346 0.01906069 0.01879629 0.01977727
  0.02561363 0.05778319]]


### Тест 2

a11 = 0.5  
a12 = 0.5  
a22 = 0.5  
a21 = 0.5  
b1(O) = 0.5  
b1(Р) = 0.5  
b2(О) = 0.51  
b2(Р) = 0.49

In [5]:
a = np.array([
    [0.5, 0.5],
    [0.5, 0.5]
])

b = [
    {'O': 0.5, 'P':0.5},
    {'O': 0.51, 'P':0.49}
]

seq = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

pi = [0.5, 0.5]

v1 = viterbi(a, b, pi, seq)
print(seq)
print(pretty_coin_ans(v1))
print(forward_backward(a, b, pi, seq))

OPOPOPOOPPPPPPPPPPOOOOOOOO
-+-+-+--++++++++++--------
[[0.4950495  0.50505051 0.4950495  0.50505051 0.4950495  0.50505051
  0.4950495  0.4950495  0.50505051 0.50505051 0.50505051 0.50505051
  0.50505051 0.50505051 0.50505051 0.50505051 0.50505051 0.50505051
  0.4950495  0.4950495  0.4950495  0.4950495  0.4950495  0.4950495
  0.4950495  0.4950495 ]
 [0.5049505  0.49494949 0.5049505  0.49494949 0.5049505  0.49494949
  0.5049505  0.5049505  0.49494949 0.49494949 0.49494949 0.49494949
  0.49494949 0.49494949 0.49494949 0.49494949 0.49494949 0.49494949
  0.5049505  0.5049505  0.5049505  0.5049505  0.5049505  0.5049505
  0.5049505  0.5049505 ]]
